# Fake News Classification


### Connect Colab to VSCode


In [ ]:
!pip install kora -q
from kora import jupyter
jupyter.start(lab=True)
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
## Import Libraries

In [1]:
import tqdm
import importlib
#@title Run this code to get started
imports = {
    'pandas':'pd',
    'numpy': 'np',
    'matplotlib.pyplot': 'plt',
    'seaborn': 'sns',
    'sklearn':None,
    'string':None,
    'nltk':None,
    'spacy':None,
    'wordcloud':None,
    'sys':None,
    'os':None}

print("Importing libraries")
for name in tqdm.tqdm(imports):
    try:
        alias = imports[name] or name
        globals()[alias] = __import__(name)
    except ImportError:
        print(f"Failed importing {name}")

# import math
# import os
# import numpy as np
# import pandas as pd

# import pickle

# import requests, io, zipfile

# Download class resources...

basepath = './dataset'
#basepath = 'drive/MyDrive/Notebooks/Fake_news_data'

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


pd.options.mode.chained_assignment = None

100%|██████████| 11/11 [00:16<00:00,  1.51s/it]


In [2]:
print("Importing NLP")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('wordnet')
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
#!python -m spacy download en_core_web_md
import en_core_web_md
text_to_nlp = en_core_web_md.load()

Importing NLP
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lolzc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lolzc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Load Data
fake_data = pd.read_csv(os.path.join(basepath, 'Fake.csv'))
true_data = pd.read_csv(os.path.join(basepath, 'True.csv'))
# Format the data
fake_data['fake'] = 1.0
true_data['fake'] = 0.0

# Convert the date column into machine-readable
# def date(val):
#     datet : datetime.datetime = parser.parse(val)
#     return [datet.year, datet.month, datet.day]

# lf = len(fake_data)
# lt = len(true_data)

# print(f'datapoints: {lt+lf}')
# print(f'% fake: {lt/lf * 100}')

dataset = pd.concat([fake_data, true_data])
del dataset['date']

# Dataset Views

In [5]:
print("Topics:\n\t", *list(set(dataset['subject'])), sep='\n\t')

dataset.head()


Topics:
	
	Government News
	left-news
	News
	worldnews
	US_News
	politics
	politicsNews
	Middle-east


,title,text,subject,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,1.0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,1.0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,1.0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,1.0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,1.0


### Generate Word cloud for fake/real news

In [ ]:
fake = True
print("Fake news Word cloud")
news_text = ''
for t in dataset[dataset['fake'] == fake]['text'].values: # form field cell
    news_text += t + ' '
    
wordcloud = WordCloud()    
wordcloud.generate_from_text(news_text)
plt.figure(figsize=(14,7))
plt.imshow(wordcloud, interpolation='bilinear')

In [ ]:
fake = False
print("Real news Word cloud")
news_text = ''
for t in dataset[dataset['fake'] == fake]['text'].values: # form field cell
    news_text += t + ' '
    
wordcloud = WordCloud()    
wordcloud.generate_from_text(news_text)
plt.figure(figsize=(14,7))
plt.imshow(wordcloud, interpolation='bilinear')

In [ ]:
fake = True
print("Fake news Title cloud")
news_text = ''
for t in dataset[dataset['fake'] == fake]['title'].values: # form field cell
    news_text += t + ' '
    
wordcloud = WordCloud()    
wordcloud.generate_from_text(news_text)
plt.figure(figsize=(14,7))
plt.imshow(wordcloud, interpolation='bilinear')

In [ ]:
fake = False
print("True news Title cloud")
news_text = ''
for t in dataset[dataset['fake'] == fake]['title'].values: # form field cell
    news_text += t + ' '
    
wordcloud = WordCloud()    
wordcloud.generate_from_text(news_text)
plt.figure(figsize=(14,7))
plt.imshow(wordcloud, interpolation='bilinear')

AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'vectors'

## Prepare data for NLP

In [ ]:
# SKIP
# if not "text_to_nlp" in globals():
#     print("Loading NLP converter")
#     text_to_nlp = en_core_web_md.load() #Prepare Spacy
# print("Tokenizing text")


# x_word2vec = np.zeros((len(dataset),), dtype=object)
# i = 0
# for text in tqdm.tqdm(dataset['text']):
#     x_word2vec[i] = text_to_nlp(text) # Array of tokens, passed to model
#     i += 1


# # print("Tokenizing titles")
# # x_titlevec = np.zeros((len(dataset),), dtype=object)
# # i = 0
# # for text in tqdm.tqdm(dataset['title']):
# #     x_titlevec[i] = text_to_nlp(text) # Array of tokens, passed to model
# #     i += 1
# # x_titlevec = np.array(x_word2vec)

In [83]:
# Memory mapper
print(*[f'{k} : {sys.getsizeof(v)} B' for (k, v) in globals().items()], sep='\n')

__name__ : 57 B
__doc__ : 113 B
__package__ : 16 B
__loader__ : 16 B
__spec__ : 16 B
__builtin__ : 72 B
__builtins__ : 72 B
_ih : 776 B
_oh : 232 B
_dh : 96 B
In : 776 B
Out : 232 B
get_ipython : 64 B
exit : 48 B
quit : 48 B
_ : 51 B
__ : 51 B
___ : 51 B
sys : 72 B
os : 72 B
_i : 722 B
_ii : 102 B
_iii : 270 B
_i1 : 1036 B
tqdm : 72 B
importlib : 72 B
imports : 640 B
name : 51 B
alias : 51 B
pd : 72 B
np : 72 B
plt : 72 B
sns : 72 B
sklearn : 72 B
string : 72 B
nltk : 72 B
spacy : 72 B
wordcloud : 72 B
basepath : 58 B
precision_recall_fscore_support : 136 B
accuracy_score : 136 B
confusion_matrix : 136 B
_i2 : 755 B
stopwords : 48 B
WordNetLemmatizer : 1064 B
word_tokenize : 136 B
train_test_split : 136 B
classification_report : 136 B
_exit_code : 16 B
en_core_web_md : 72 B
text_to_nlp : 48 B
_i3 : 621 B
datetime : 72 B
parser : 72 B
_i4 : 724 B
_i5 : 120 B
_i6 : 756 B
_i7 : 621 B
date : 136 B
lf : 28 B
lt : 28 B
_i8 : 724 B
x_word2vec : 359280 B
i : 28 B
text : 1387 B
_i9 : 120 B
_i10

In [56]:
# x_word2vec = np.concat(np.load(os.path.join("E:\\Desktop\\Dataset", 'word_vecs1')), np.load(os.path.join("E:\\Desktop\\Dataset", 'word_vecs')) # Necessary!!!

In [57]:
# # Search for first zero
# l = 0
# r = len(x_word2vec)-1
# m = 0
# while l < r:
#     m = (l + m + 1)//2
#     if x_word2vec[m] == 0:
#         r = m
#     else:
#         l = m+1
# print(m)

44896


# Load ML libraries



In [4]:
import tensorflow as tf
from tensorflow import keras


def ragged_concat(arr, axis=1):
    return tf.RaggedTensor.from_row_lengths(tf.concat(arr, axis=axis), row_lengths=tuple(len(i) for i in arr))

conv3 = keras.layers.Conv1D(8, kernel_size=3, padding='same')
conv2 = keras.layers.Conv1D(4, kernel_size=2, padding='same')
conv5 = keras.layers.Conv1D(4, kernel_size=4, padding='same')


l = [conv5(tf.zeros((1, 12, 300))), conv3(tf.zeros((1, 12, 300))), conv2(tf.zeros((1, 12, 300)))]
# Shape is (Batch, Word Vecs, Word Vec size)
# 'same' padding fixes the word vec size
# We have different filter sizes
# (Last one)
tf.concat(l, 2)

## Build the model

In [6]:

class MultiKernel1D(keras.layers.Layer):
    """Supports using multiple kernel sizes for convoltion on text"""
    def __init__(self, filters:tuple=(100, 100, 100), kernels:tuple=(3,4,5), input_shape=(None, 300)):
        super(MultiKernel1D, self).__init__()
        self.kernels = [keras.layers.Conv1D(filter, kernel, input_shape=input_shape, padding='same') for (filter, kernel) in zip(filters, kernels)]

    def call(self, inputs, training=None):
        # Call each of the kernels
        # Output should be like
        # Input shape is a tensor of size (A, 300)
        # Which means a variable vector of 300-vectors
        # Which represent word embeddings

        # Each convolution outputs a vector that is (A, filters)
        # We need to combine them into a ragged tensor
        # [
        #   C1: [F1: [?, ?, ...], F2: [?, ?, ...], ...]
        #   C2: [F1: [?, ?, ...], F2: [?, ?, ...], ...]
        #   C3: [F1: [?, ?, ...], F2: [?, ?, ...], ...]
        # ]
        # 3 dimensions, ragged middle vector
        return tf.concat([kernel(inputs) for kernel in self.kernels], -1)

        #     Conv 2
        #  /        /F1
        # Conv 1     F2
        # 0 | F1 F2 ... F3
        # 1 | F1 F2  F4 
        # 2 | F1 F2  /
        # 3 | F1 F2 /
        # 4 | F1 F2
        # So 3-d Tensor, or 4-d with batches


        

class NewsNet(keras.Model):
    def __init__(self):
        super(NewsNet, self).__init__()
        self.kernels = MultiKernel1D()

        self.sequential = keras.layers.LSTM(30)
        self.dense10 = keras.layers.Dense(10)
        self.predictor = keras.layers.Dense(1)
    
    def call(self, x, training=None):

        x = self.kernels(x)
        x = self.sequential(x)
        x = self.dense10(x)
        return self.predictor(x)[0]


In [8]:
newsNet = NewsNet()
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
x_train, x_test, y_train, y_test = train_test_split(dataset['text'], dataset['fake'].to_numpy(), train_size=0.07)

loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)


# Training the model



In [19]:
## Note: Rerunning this cell uses the same model variables
global printed
model = newsNet
printed = False
def print_remove(s, *a, **k):
    global printed
    if (printed):
        print('\r' + s, *a, **k)
    else:
        print(s, *a, **k)

@tf.function(input_signature=(tf.TensorSpec(shape=[1, None, 300], dtype=tf.float32), tf.TensorSpec(shape=(1), dtype=tf.float32)))
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value


train_acc_metric = keras.metrics.BinaryAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()
@tf.function(input_signature=(tf.TensorSpec(shape=[1, 1, None, 300], dtype=tf.int32),))
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)


# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

import time

epochs = 20
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()
    step = 0
    # Iterate over the batches of the dataset.
    for text, fake in tqdm.tqdm(zip(x_train, y_train)):
        printed = False
        x_batch_train = tf.constant([[np.array(x.vector) for x in text_to_nlp(text)]])
        y_batch_train = [fake]
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print_remove(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
        step += 1

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print_remove("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    step = 0
    for text, fake in tqdm.tqdm(zip(x_test, y_train)):
        x_batch_val = tf.constant([[x.vector for x in text_to_nlp(text)]])
        y_batch_val = np.reshape(list(fake), shape=(1, 1))
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print_remove("Validation acc: %.4f" % (float(val_acc),))
    print_remove("Time taken: %.2fs" % (time.time() - start_time))





  0%|          | 0/3142 [00:00<?, ?it/s]
Start of epoch 0


  0%|          | 1/3142 [00:02<2:19:58,  2.67s/it]Training loss (for one batch) at step 0: 0.0026


  0%|          | 2/3142 [00:03<1:08:58,  1.32s/it]

  0%|          | 3/3142 [00:03<41:12,  1.27it/s]  

  0%|          | 4/3142 [00:03<33:52,  1.54it/s]

  0%|          | 5/3142 [00:03<28:18,  1.85it/s]

  0%|          | 6/3142 [00:04<24:56,  2.10it/s]

  0%|          | 7/3142 [00:04<22:53,  2.28it/s]

  0%|          | 8/3142 [00:05<25:52,  2.02it/s]

  0%|          | 9/3142 [00:05<25:02,  2.09it/s]

  0%|          | 10/3142 [00:06<22:13,  2.35it/s]

  0%|          | 11/3142 [00:06<18:19,  2.85it/s]

  0%|          | 12/3142 [00:06<16:38,  3.13it/s]

  0%|          | 13/3142 [00:06<16:13,  3.22it/s]

  0%|          | 15/3142 [00:07<17:15,  3.02it/s]

  1%|          | 16/3142 [00:07<18:07,  2.88it/s]

  1%|          | 17/3142 [00:08<15:42,  3.32it/s]

  1%|          | 18/3142 [00:08<19:37,  2.65it/s]

  1%|          | 19/3142 [

KeyboardInterrupt: 

## Exercise 4 



In [ ]:
train_y_pred = baseline_model.predict(train_X)
print('Train accuracy', accuracy_score(train_y, train_y_pred))

We can see that we are not doing very well, but we are doing better than 50%. We can do the same for the val data to see how we are doing on unseen data, which is more valuable for us if we want to make predictions on new websites. Fill in the code below to evaluate val accuracy!

We appear to be doing a bit worse on the val data, not much better than chance. To better understand the performance of our binary classification model, we should seek to better understand the mistakes that it is making. Specifically, when our model makes a mistake (about 40% of the time), are these mistakes false negatives or false positives?

## Confusion Matrix 

To answer these questions, we produce and analyze the confusion matrix. The confusion matrix is a matrix that shows the following:

![Confusion Matrix](https://cdn-images-1.medium.com/max/1600/1*Z54JgbS4DUwWSknhDCvNTQ.png)

where the terms mean

* TP (True Positive) = You predicted positive (fake in our case, since fake has a label of 1) and it’s true.
* FP (False Positive) = You predicted positive and it’s false.
* FN (False Negative) = You predicted negative and it’s false.
* TN (True Negative) = You predicted negative and it’s true.


###Common Metrics

From the confusion matrix, we can extract commonly used metrics like precision (TP/(TP + FP)) and recall (TP/(TP + FN)). 

* Precision quantifies how often the things we classify as positive are actually positive. For our task, this measures what fraction of the sites we classify as fake are actually fake. 
* Recall quantifies what fraction of actually positive examples we classify as positive. In our case, this is the fraction of fake news websites that we actually identify as fake.

Finally, a useful score to summarize both precision and recall is the F-1 score. This is just a simple function (the harmonic mean) of precision and recall, shown in the summary below:

<img src="https://datascience103579984.files.wordpress.com/2019/04/capture3-24.png" width="400" height="200"></img>

##Exercise 5 |  Using the Confusion Matrix 

Run the cell below to create the confusion matrix for our own model. 

In [ ]:
print('Confusion matrix:')
print(confusion_matrix(val_y, val_y_pred))

A Confusion Matrix can quickly tell you how well your model is doing. The primary way to figure this out is to calculate the Error Rate. 

The Error Rate is:   (FP) + (FN)) / (TP + FP + FN + TN).

This is just all the false predictions (False Negative + False Positive) divided by all the predictions added together.  

Use the Confusion Matrix we just created to calculate the Error Rate for our model. 

In [ ]:
### YOUR CODE HERE ###

### END CODE HERE ###

## Exercise 6

In [ ]:
print(val_y_pred)

We can see that we have many false negatives, and not as many false positives. Why is this the case? If we print out *val_y_pred*, we can see that our model is mostly predicting 0's (websites are real).

What fraction of predictions in *val_y_pred* are 1's? Hint: you may find *np.mean* useful.

In [ ]:
### YOUR CODE HERE ###

### END CODE HERE ###

Why so many 0's? The only information we are giving our model is its domain name extension. It's natural that the model would learn that websites with ".biz" extensions are unlikely to be reliable news websites, but it is still the case that most websites in the dataset (fake and real) have ".com" extensions. Thus, our model will misclassify many fake news websites with ".com" extensions as real. 

In [ ]:
prf = precision_recall_fscore_support(val_y, val_y_pred)

print('Precision:', prf[0][1])
print('Recall:', prf[1][1])
print('F-Score:', prf[2][1])

Again, the precision and recall metrics suggest that when we classify a website as fake, we are usually right, but we are not doing great at classifying these websites as fake frequently enough.

##Using Keywords for a Stronger Baseline 

The key problem with our model in its current state is that it simply does not have enough information. This should not be a surprise–it was pretty unlikely in the first place that domain name extensions would be enough. If you like, feel free to add a few more extensions in the “featurizer” above and re-run all the code for evaluation–you'll find it doesn't make much of a difference.
Where can we get more information about webpages? From the HTML! Remember that the HTML contains all of the text and structure of a webpage. If we cleverly choose features from the HTML to feed into our logistic regression model, we will drastically improve our performance. We saw yesterday that probing hypotheses related to the counts of hypotheses words produced interesting results, and we will continue in this direction today to produce a model that leverages these differences in word frequencies.


## Exercise 7: Instructor-Led Discussion on Better Input Features



The below code introduces a better featurizer that counts the number of keywords (normalized using the *log* function) in the HTML. Normalizing the counts is a trick that prevents the featurized values from becoming too extreme. Read the code and make sure you understand what it is doing. Then add "sports" and "finance" as additional keywords to expand our model.

**Run the below code and discuss what it is doing as a class. Add in additional keywords to further expand our model as you see fit.**





##Exercise 8



Let's run and evaluate the above featurizer. Add in code to fit the model, compute train accuracy, val accuracy, val confusion matrix, and val precision, recall, and F1-Score, just as before.

In [ ]:
train_X, train_y, feature_descriptions = prepare_data(train_data, keyword_featurizer)
val_X, val_y, feature_descriptions = prepare_data(val_data, keyword_featurizer)

print('Number of features per example:', len(train_X[0]))
print('Feature descriptions:')
print(feature_descriptions)
print()
  
baseline_model = LogisticRegression()

### YOUR CODE HERE ###

### END CODE HERE ###

## Interpreting our Model



### Instructor-Led Discussion: Interpreting Input Variables

As mentioned earlier, a key motivation for using a simpler model is interpretability.

We've learned that the prediction of a logistic regression classifier is just the output of a multiplication with model weights, followed by a non-linear transformation (sigmoid). Because the sigmoid function is always increasing (monotonic) on its domain (see below), we know that if the dot product (or multiplication of vectors) between model weights and input features is large, then the output prediction will be closer to 1. If the dot product is small, then the output prediction will be closer to 0.

![Sigmoid](https://cdn-images-1.medium.com/max/2400/1*RqXFpiNGwdiKBWyLJc_E7g.png)

Thus, the weights corresponding to features tell us whether the features are important in the classification. If the weight corresponding to the feature ".net domain" has a large positive value, then websites with ".net" domains are more likely to be classified as fake (since fake has label 1). If it has a large negative value, then these websites are more likely to be classified as real. If it has value close to 0, then the feature may not be useful (at least, it may not be useful given that the other features are present).


###Using Feature Descriptions

Let's see what weights our model learned. The code below uses *feature_descriptions* and the weights, or coefficients, of the model and sorts them in ascending order.

In [ ]:
sorted(zip(feature_descriptions, baseline_model.coef_[0].tolist()), key=lambda x: x[1])

## Exercise 9

Answer the following questions:

* What features have positive weight (most predictive of being fake)? What does that indicate?
* Which ones have negative weight (most predictive of being real)? What does that indicate?
* Which ones have close to 0 weight? 
* Are there any feature weights that surprise you? 
* Try coming up with explanations for why the feature weights are the way they are. Does this help you come up with new feature ideas? (~15 minutes)

In [ ]:
'''
- 
'''

## Instructor-Led Discussion: Final Interpretation of Inputs

##Exercise 10 |  Final Baseline

Finally, play around with the last few cells, adding more keywords and domain names to see how the results change. Note that "keywords" can be a variety of things: English words, English phrases (spaces are allowed), HTML tags, and any other string present in HTML. Also notice how the weights on different features vary–you may observe some interesting effects. When you are done, run the cell below to run evaluations again!

In [ ]:
train_y_pred = baseline_model.predict(train_X)
print('Train accuracy', accuracy_score(train_y, train_y_pred))

val_y_pred = baseline_model.predict(val_X)
print('Val accuracy', accuracy_score(val_y, val_y_pred))

print('Confusion matrix:')
print(confusion_matrix(val_y, val_y_pred))

prf = precision_recall_fscore_support(val_y, val_y_pred)

print('Precision:', prf[0][1])
print('Recall:', prf[1][1])
print('F-Score:', prf[2][1])

Congratulations on completing this notebook. Looking at the results of our final baseline, you may be surprised this approach is working at all–after all, our model is still barely looking at the content of websites. We will further explore the issue of modeling the content of websites tomorrow, but as a result of our efforts today, we now know that we can make progress with a relatively simple approach!